In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import time 


gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 
    try:
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

import sys
from pathlib import Path

sys.path.append(str(Path('..')))

from sionna.fec.ldpc import QLDPCBPDecoder, Feedback_GNN, load_weights
from sionna.fec.ldpc import Sandwich_BP_GNN_Evaluation_Model 
from sionna.fec.ldpc import *
from sionna.utils.plotting import PlotBER

from tqdm.notebook import tqdm

Number of GPUs available : 1
Only GPU number 0 used.


In [3]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24

bs = tf.constant(5000) # please adjust bs to best fit into your gpu
max_iter = tf.constant(200000)

n = tf.constant(code.N)
cn_z = tf.constant(code.hz.shape[0])
cn_x = tf.constant(code.hx.shape[0])

G = Feedback_GNN(code=code, 
                 num_msg_dims=tf.constant(20),
                 num_hidden_units=tf.constant(40),
                 num_mlp_layers=2,
                 reduce_op="mean",      
                 activation="tanh",
                 use_bias=True)
G((tf.zeros((bs, n, 3)), tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs)), 
                  tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs))))
load_weights(G, f"../sionna/fec/ldpc/weights/feedback_GNN_n882_k24_wt_4_60_iter_64_16_mixed.npy")     #### change here

ber_plot = PlotBER()

In [4]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(1.0)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)

model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*3, [G]*3, num_layers=4)
p = np.arange(0.04, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100, # stop sim after 2000 bit errors
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G)",
              soft_estimates=True,
              max_mc_iter=max_iter,
              early_stop=True, 
              add_bler=True, 
              show_fig=False, 
              qldpc=True,
              forward_keyboard_interrupt=False)

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 4.7500e-01 | 4.7500e-01 |        2375 |         2375 |        5000 |         8.4 |reached target block errors
     0.13 | 2.3240e-01 | 2.3240e-01 |        1162 |         1162 |        5000 |         0.5 |reached target block errors
     0.12 | 7.9200e-02 | 7.9200e-02 |         396 |          396 |        5000 |         0.5 |reached target block errors
     0.11 | 1.8500e-02 | 1.8500e-02 |         185 |          185 |       10000 |         0.9 |reached target block errors
      0.1 | 3.7667e-03 | 3.7667e-03 |         113 |          113 |       30000 |         2.7 |reached target block errors
     0.09 | 5.8824e-04 | 5.8824e-04 |         100 |          100 |      170000 |        15.5 |reached target block errors
     0.08 | 7.5472e-05 | 7.

(<tf.Tensor: shape=(11,), dtype=float64, numpy=
 array([4.75000000e-01, 2.32400000e-01, 7.92000000e-02, 1.85000000e-02,
        3.76666667e-03, 5.88235294e-04, 7.54716981e-05, 1.05540897e-05,
        2.21606648e-06, 4.56516777e-07, 1.98491465e-07])>,
 <tf.Tensor: shape=(11,), dtype=float64, numpy=
 array([4.75000000e-01, 2.32400000e-01, 7.92000000e-02, 1.85000000e-02,
        3.76666667e-03, 5.88235294e-04, 7.54716981e-05, 1.05540897e-05,
        2.21606648e-06, 4.56516777e-07, 1.98491465e-07])>)

In [6]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(1.0)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)

model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*5, [G]*5, num_layers=6)
p = np.arange(0.04, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100, 
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G,G,G)",
              soft_estimates=True,
              max_mc_iter=20000,
              early_stop=True, 
              add_bler=True, 
              show_fig=False, 
              qldpc=True,
              forward_keyboard_interrupt=False)


        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 4.7420e-01 | 4.7420e-01 |        2371 |         2371 |        5000 |        13.7 |reached target block errors
     0.13 | 2.2440e-01 | 2.2440e-01 |        1122 |         1122 |        5000 |         0.7 |reached target block errors
     0.12 | 6.7400e-02 | 6.7400e-02 |         337 |          337 |        5000 |         0.7 |reached target block errors
     0.11 | 1.5200e-02 | 1.5200e-02 |         152 |          152 |       10000 |         1.4 |reached target block errors
      0.1 | 2.4000e-03 | 2.4000e-03 |         108 |          108 |       45000 |         6.0 |reached target block errors
     0.09 | 2.6579e-04 | 2.6579e-04 |         101 |          101 |      380000 |        50.6 |reached target block errors
     0.08 | 5.1680e-05 | 5.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     0.04 | 8.0000e-08 | 8.0000e-08 |           8 |            8 |   100000000 |     13308.7 |reached max iter       


(<tf.Tensor: shape=(11,), dtype=float64, numpy=
 array([4.74200000e-01, 2.24400000e-01, 6.74000000e-02, 1.52000000e-02,
        2.40000000e-03, 2.65789474e-04, 5.16795866e-05, 8.83002208e-06,
        1.36472194e-06, 1.90000000e-07, 8.00000000e-08])>,
 <tf.Tensor: shape=(11,), dtype=float64, numpy=
 array([4.74200000e-01, 2.24400000e-01, 6.74000000e-02, 1.52000000e-02,
        2.40000000e-03, 2.65789474e-04, 5.16795866e-05, 8.83002208e-06,
        1.36472194e-06, 1.90000000e-07, 8.00000000e-08])>)

## Factor 0.8

In [3]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(0.8)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)

model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*3, [G]*3, num_layers=4)
p = np.arange(0.06, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100, 
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G)",
              soft_estimates=True,
              max_mc_iter=max_iter,
              early_stop=True, 
              add_bler=True, 
              show_fig=False, 
              qldpc=True,
              forward_keyboard_interrupt=False)

Tracing MLP for num_units=ListWrapper([<tf.Tensor: shape=(), dtype=int32, numpy=40>, <tf.Tensor: shape=(), dtype=int32, numpy=20>]), input shape=(None, 2646, 4), input type=<class 'tensorflow.python.framework.ops.Tensor'>.
Tracing MLP for num_units=ListWrapper([<tf.Tensor: shape=(), dtype=int32, numpy=40>, <tf.Tensor: shape=(), dtype=int32, numpy=20>]), input shape=(None, 2646, 4), input type=<class 'tensorflow.python.framework.ops.Tensor'>.
Tracing reduce msg in update VN embedding for msg shape=(None, 2646, 20)
Tracing MLP for num_units=ListWrapper([<tf.Tensor: shape=(), dtype=int32, numpy=40>, <tf.Tensor: shape=(), dtype=int32, numpy=20>]), input shape=(None, 882, 43), input type=<class 'tensorflow.python.framework.ops.Tensor'>.
        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 3.402

(<tf.Tensor: shape=(9,), dtype=float64, numpy=
 array([3.40200000e-01, 1.39400000e-01, 3.90000000e-02, 8.73333333e-03,
        1.66153846e-03, 3.07692308e-04, 6.09756098e-05, 1.27307447e-05,
        0.00000000e+00])>,
 <tf.Tensor: shape=(9,), dtype=float64, numpy=
 array([3.40200000e-01, 1.39400000e-01, 3.90000000e-02, 8.73333333e-03,
        1.66153846e-03, 3.07692308e-04, 6.09756098e-05, 1.27307447e-05,
        0.00000000e+00])>)

In [5]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(0.8)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)

model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*5, [G]*5, num_layers=6)
p = np.arange(0.07, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100,
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G,G,G)",
              soft_estimates=True,
              max_mc_iter=max_iter,
              early_stop=True,
              add_bler=True,
              show_fig=False,
              qldpc=True,
              forward_keyboard_interrupt=False)

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 3.3040e-01 | 3.3040e-01 |        1652 |         1652 |        5000 |        13.7 |reached target block errors
     0.13 | 1.3360e-01 | 1.3360e-01 |         668 |          668 |        5000 |         0.7 |reached target block errors
     0.12 | 3.6800e-02 | 3.6800e-02 |         184 |          184 |        5000 |         0.7 |reached target block errors
     0.11 | 8.2000e-03 | 8.2000e-03 |         123 |          123 |       15000 |         2.0 |reached target block errors
      0.1 | 1.3867e-03 | 1.3867e-03 |         104 |          104 |       75000 |        10.0 |reached target block errors
     0.09 | 2.4337e-04 | 2.4337e-04 |         101 |          101 |      415000 |        55.5 |reached target block errors
     0.08 | 5.8651e-05 | 5.

(<tf.Tensor: shape=(8,), dtype=float64, numpy=
 array([3.30400000e-01, 1.33600000e-01, 3.68000000e-02, 8.20000000e-03,
        1.38666667e-03, 2.43373494e-04, 5.86510264e-05, 1.01781170e-05])>,
 <tf.Tensor: shape=(8,), dtype=float64, numpy=
 array([3.30400000e-01, 1.33600000e-01, 3.68000000e-02, 8.20000000e-03,
        1.38666667e-03, 2.43373494e-04, 5.86510264e-05, 1.01781170e-05])>)